In [2]:
from simple_salesforce import Salesforce
import pandas as pd
import os

In [3]:
# connection to Salesforce
SF = Salesforce("iet@detroitmi.gov", os.environ['SF_PASS'], os.environ['SF_TOKEN'])

In [4]:
# Helper functions
def display_query(query):
    """Pretty-print a Salesforce query."""
    print(json.dumps(SF.query(query), indent=1))
    return SF.query_all(query)

def display_obj(obj):
    """Pretty-print a Salesforce object (ordered dict)"""
    print(json.dumps(obj, indent=1))
    
def process_record(record, fieldmap):
    """Create a new record from Salesforce via a field mapping."""
    data = {}
    for k, v in fieldmap.items():
        try:
            data[k] = record[v]
        except: 
            print("no match on field: {}".format(v))
    return data

In [11]:
next_thirty = SF.query_all("""
                           Select Id, 
                            Parcel_ID__c, 
                            Address__c, 
                            DEMO_Planned_Knock_Down_Date__c
                           from Case 
                            WHERE DEMO_Planned_Knock_Down_Date__c = NEXT_N_DAYS:30
                            ORDER BY DEMO_Planned_Knock_Down_Date__c ASC
                            """)

mapping = {
    'Parcel ID': 'Parcel_ID__c',
    'Address': 'Address__c',
    'Projected Demolition Date': 'DEMO_Planned_Knock_Down_Date__c',
}

In [12]:
# get an array of processed SF rows
rows = [process_record(r, mapping) for r in next_thirty['records']]

In [19]:
demos_df = pd.DataFrame(rows)
print(demos_df.shape)
print(demos_df.dtypes)

(113, 3)
Address                      object
Parcel ID                    object
Projected Demolition Date    object
dtype: object


In [20]:
from sodapy import Socrata
# connection to Socrata
socrata_client = Socrata("data.detroitmi.gov", os.environ['SODA_TOKEN'], os.environ['SODA_USER'], os.environ['SODA_PASS'])

In [25]:
# columns for Next 30 Days demolition dataset
columns = [
    {"fieldName": "parcelnumb", "name": "Parcel ID", "dataTypeName": "text"},
    {"fieldName": "demo_date", "name": "Projected Demolition Date", "dataTypeName": "date"},
    {"fieldName": "address", "name": "Address", "dataTypeName": "text"},
]

# tags for Next 30 Days demolition dataset
tags = ["DLBA", "demolitions"]

In [29]:
# make the dataset
dataset = socrata_client.create("Demolitions Next 30 Days", 
                      description="Demolitions happening in the next 30 days",
                      columns=columns,
                      row_identifier="parcelnumb",
                      tags=tags,
                      category="Government")

In [30]:
# upsert rows from Salesforce
socrata_client.upsert(dataset['id'], rows)

{'By RowIdentifier': 113,
 'By SID': 0,
 'Errors': 0,
 'Rows Created': 113,
 'Rows Deleted': 0,
 'Rows Updated': 0}